# Merging 1 - orders_clean and orders_products_prior_clean

### List of contents
    1. Importing file
    2. Checking for mixed type column
    3. Merging files
    4. Missing Values?
    5. Dropping unwanted columns
    6. Exporting 

## Importing libraries

In [3]:
import pandas as pd
import numpy as np
import os

## Creating path for folder

In [6]:
path = r'/Users/ceciliamoura/Desktop/Career Foundry/Achievement4/Instacart Basket Analysis'

## 1. Importing files

In [42]:
# Importing orders_clean

In [14]:
orders = pd.read_csv(os.path.join (path, 'Data', 'Final Prepared Data', 'orders_clean.csv'), index_col = False)

In [16]:
orders.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_time,days_since_prior_order
0,0,2539329,1,1,2,8,0
1,1,2398795,1,2,3,7,15
2,2,473747,1,3,3,12,21
3,3,2254736,1,4,4,7,29
4,4,431534,1,5,4,15,28


In [25]:
orders.shape

(3421083, 7)

In [44]:
# Importing orders_products_prior_clean

prior = pd.read_csv(os.path.join (path, 'Data', 'Final Prepared Data', 'orders_products_prior_clean.csv'), index_col = False)

In [48]:
prior.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered
0,0,2,33120,1,1
1,1,2,28985,2,1
2,2,2,9327,3,0
3,3,2,45918,4,1
4,4,2,30035,5,0


In [50]:
prior.shape

(32434489, 5)

## 2. Checking for mixed type column

In [35]:
# orders

for col in orders.columns.tolist():
  weird = (orders[[col]].map(type) != orders[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (orders[weird]) > 0:
    print (col)

In [ ]:
# no mixed type column found

In [37]:
# prods

for col in prods.columns.tolist():
  weird = (prods[[col]].map(type) != prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (prods[weird]) > 0:
    print (col)

In [39]:
# no mixed type column found

In [19]:
prior.dtypes

order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object

## 3. Merging files

In [84]:
# key = ['order_id']
# left file = prior (32434489 observations)
# right file = orders (3421083 observations)
# will use inner merge since I want a full matching between the files

ords_prods_merged = prior.merge(orders, on = 'order_id', how = 'inner', indicator = True)

In [86]:
#checking new merged file
ords_prods_merged.shape

(32434489, 12)

In [88]:
ords_prods_merged.head()

,Unnamed: 0_x,order_id,product_id,add_to_cart_order,reordered,Unnamed: 0_y,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,_merge
0,0,2,33120,1,1,3355525,202279,3,5,9,8,both
1,1,2,28985,2,1,3355525,202279,3,5,9,8,both
2,2,2,9327,3,0,3355525,202279,3,5,9,8,both
3,3,2,45918,4,1,3355525,202279,3,5,9,8,both
4,4,2,30035,5,0,3355525,202279,3,5,9,8,both


In [90]:
ords_prods_merged['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [92]:
# Double checking merge using outer merge

ords_prods_outter_merged = prior.merge(orders, on = 'order_id', how = 'left', indicator = True)

In [94]:
ords_prods_outter_merged.shape

(32434489, 12)

In [96]:
ords_prods_outter_merged['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

## 4. Missing values?

In [99]:
ords_prods_merged.isnull().sum()

Unnamed: 0_x              0
order_id                  0
product_id                0
add_to_cart_order         0
reordered                 0
Unnamed: 0_y              0
user_id                   0
order_number              0
order_day_of_week         0
order_time                0
days_since_prior_order    0
_merge                    0
dtype: int64

In [32]:
# No missing values found

## 5. Dropping unwanted columns

In [104]:
ords_prods_merged = ords_prods_merged.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y', '_merge'])

In [110]:
# checking procedure

ords_prods_merged.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'order_number', 'order_day_of_week', 'order_time',
       'days_since_prior_order'],
      dtype='object')

In [112]:
ords_prods_merged.shape

(32434489, 9)

In [116]:
ords_prods_merged.dtypes

order_id                  int64
product_id                int64
add_to_cart_order         int64
reordered                 int64
user_id                   int64
order_number              int64
order_day_of_week         int64
order_time                int64
days_since_prior_order    int64
dtype: object

In [118]:
# I will switch columns: order_id, product_id and user_id into object after final merge. 

## 6. Exporting combined file

In [123]:
ords_prods_merged.to_pickle(os.path.join(path, 'Data','Final Prepared Data', 'orders_products_combined.pkl'))